#### Compute the Regions of Interest for each image
using the Matlab Selective Search.

##### Imports

In [66]:
import os
import utils
import importlib
importlib.reload(utils)
from utils import *
from scipy import io
import numpy as np
import time
import PIL
import random

from selective_search.selective_search_ijcv_with_python import selective_search
importlib.reload(selective_search)
from selective_search.selective_search_ijcv_with_python.selective_search \
    import get_windows

from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

##### Load the filenames make the filepath an absolute path to the image.

In [2]:
filenames = load_array('intermediate/voc/filenames.bcolz')
images_dir = '/home/slav/DL/RCNN'
fullpaths = [os.path.join(images_dir, filename) 
                       for filename in filenames]
len(fullpaths)

17125

##### Prepare a batch of 10 images and send it to the matlab function.

In [3]:
matlab_path = '/usr/local/MATLAB/R2016a/bin/matlab'
batch_size = 10
def get_ss_regions(i):
    print('@{} '.format(i), end='')
    return [i, get_windows(fullpaths[i:i+batch_size], 
                       'selective_search_rcnn', 
                       matlab_path)]

##### Run the above function on all batches.

In [ ]:
%%time
num_samples = len(fullpaths)
regions = []
indexes = range(0, num_samples, batch_size)
with ThreadPoolExecutor(max_workers=4) as executor:
    regions = executor.map(get_ss_regions, indexes)

##### Save

In [135]:
regions_computed = list(regions)
len(regions_computed)

1712

In [136]:
save_array('intermediate/voc/ss_matlab_computed_regions.bcolz', list(regions_computed))

In [4]:
regions_computed = load_array('intermediate/voc/ss_matlab_computed_regions.bcolz')

##### Group regions by image
The regions are currently grouped in batches. We should group them by the image they belong to.

In [ ]:
regions_ordered = []
for regions_batch in regions_computed:
    for image_regions in regions_batch:
        regions_ordered.append(image_regions)
len(regions_ordered)

##### Review

In [ ]:
img_index = 17124
filename = filenames[img_index]
rois = regions_ordered[img_index]
image = PIL.Image.open(filename) #.resize((500, 500))
rois_at_once = 50
for index in range(0, len(rois), rois_at_once):
    display_image_regions(image, rois[index:index+rois_at_once])